# Parkinson's Mouse Tutorial - Diversity

Run this notebook in `qiime2-2021.11`.

Continuing the [pd-mouse tutorial](https://docs.qiime2.org/2021.11/tutorials/pd-mice/), [here](https://docs.qiime2.org/2021.11/tutorials/pd-mice/#diversity-analysis).

*Note, we'll use the output data from the "de novo" phylogeny steps. You can replace the inputs with the fragment-insertion outputs and see how these compare.*

In [ ]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [ ]:
getcwd()

In [ ]:
chdir('../processed')
getcwd()

## Summarize table

In [ ]:
! qiime feature-table summarize \
    --i-table ./table-no-ecmu-hits.qza \
    --o-visualization ./table-no-ecmu-hits.qzv \
    --m-sample-metadata-file metadata.tsv

## Rarefaction

See [alpha-rarefaction-and-selecting-a-rarefaction-depth](https://docs.qiime2.org/2021.11/tutorials/pd-mice/#alpha-rarefaction-and-selecting-a-rarefaction-depth).

In [ ]:
! qiime diversity alpha-rarefaction \
    --i-table ./table-no-ecmu-hits.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./alpha_rarefaction_curves.qzv \
    --p-min-depth 10 \
    --p-max-depth 4996 \
    --p-steps 20

In [ ]:
q2.Visualization.load('alpha_rarefaction_curves.qzv')

## Core diversity pipeline

This is a pipeline that will run a suite of alpha and beta diversity analysis in one go.

In [ ]:
! qiime diversity core-metrics-phylogenetic \
    --i-table ./table-no-ecmu-hits.qza \
    --i-phylogeny ./mafft-fasttree-output/rooted_tree.qza \
    --m-metadata-file ./metadata.tsv \
    --p-sampling-depth 2000 \
    --output-dir ./core-metrics-results

In [ ]:
q2.Visualization.load('./core-metrics-results/weighted_unifrac_emperor.qzv')

**Let's create a biplot**

This will be silimar to the DEICODE output, wchich we'll see later. In this case we'll make a biplot with weighted unifrac.

That is map the top feature loadings onto our plot. *Note: we must use the rarefied feature-table produced from `core-metrics-phylogenetic`*

In [ ]:
# convert to relative frequency
! qiime feature-table relative-frequency \
    --i-table ./core-metrics-results/rarefied_table.qza \
    --o-relative-frequency-table ./core-metrics-results/rarefied_table_rf.qza

In [ ]:
# prepare biplot
! qiime diversity pcoa-biplot \
    --i-pcoa ./core-metrics-results/weighted_unifrac_pcoa_results.qza  \
    --i-features ./core-metrics-results/rarefied_table_rf.qza \
    --o-biplot ./core-metrics-results/weighted_unifrac_pcoa_results_biplot.qza

In [ ]:
! qiime emperor biplot \
    --i-biplot ./core-metrics-results/weighted_unifrac_pcoa_results_biplot.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --m-feature-metadata-file ./taxonomy.qza \
    --p-number-of-features 8 \
    --o-visualization ./core-metrics-results/weighted_unifrac_pcoa_results_biplot.qzv

In [ ]:
q2.Visualization.load('./core-metrics-results/weighted_unifrac_pcoa_results_biplot.qzv')

### Alpha group sig

Use the output from above to perform further analyses.

In [ ]:
! qiime diversity alpha-group-significance \
    --i-alpha-diversity ./core-metrics-results/faith_pd_vector.qza \
    --m-metadata-file ./metadata.tsv \
    --o-visualization ./core-metrics-results/faiths_pd_statistics.qzv

In [ ]:
q2.Visualization.load('./core-metrics-results/faiths_pd_statistics.qzv')

In [ ]:
! qiime longitudinal anova \
    --m-metadata-file ./core-metrics-results/faith_pd_vector.qza \
    --m-metadata-file ./metadata.tsv \
    --p-formula 'faith_pd ~ genotype * donor_status' \
    --o-visualization ./core-metrics-results/faiths_pd_anova.qzv

In [ ]:
q2.Visualization.load('./core-metrics-results/faiths_pd_anova.qzv')

### Beta group sig

Use the output from above to perform further analyses.

In [ ]:
!qiime diversity beta-group-significance \
    --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file metadata.tsv \
    --m-metadata-column donor \
    --o-visualization core-metrics-results/unweighted-unifrac-donor-significance.qzv

!qiime diversity beta-group-significance \
    --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file metadata.tsv \
    --m-metadata-column donor \
    --o-visualization core-metrics-results/weighted-unifrac-donor-significance.qzv

In [ ]:
q2.Visualization.load('core-metrics-results/unweighted-unifrac-donor-significance.qzv')

In [ ]:
q2.Visualization.load('core-metrics-results/weighted-unifrac-donor-significance.qzv')

## Empress

[GitHub repo](https://github.com/biocore/empress)

In [ ]:
!qiime empress community-plot \
    --i-tree ./mafft-fasttree-output/rooted_tree.qza \
    --i-feature-table ./core-metrics-results/rarefied_table.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --m-feature-metadata-file ./taxonomy.qza \
    --i-pcoa ./core-metrics-results/weighted_unifrac_pcoa_results.qza \
    --p-filter-extra-samples \
    --o-visualization ./core-metrics-results/community-tree-viz.qzv

In [ ]:
q2.Visualization.load('./community-tree-viz.qzv')